# Análise Estratégica de Localização CD Magalu - Dashboard Interativo
## Comparação: Recife vs Salvador

Este notebook apresenta visualizações interativas dos resultados da análise multicritério para seleção da localização do novo Centro de Distribuição do Magalu no Nordeste.

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
import json
import sys
import os

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

# Adicionar diretório src ao path
sys.path.append('../src')
sys.path.append('../config')

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# Executar análise principal
import sys
sys.path.append('..')
from main_analysis import MagaluCDAnalyzer

print("🚀 Iniciando análise completa...")
analyzer = MagaluCDAnalyzer()
resultados = analyzer.execute_complete_analysis()
print("✅ Análise concluída!")

## 1. Visualização dos Scores MCDA

In [ ]:
# Extrair dados MCDA
mcda_data = resultados['mcda']
scores = mcda_data['resultado_principal']['scores']
criterios = mcda_data['criterios_utilizados']
pesos = mcda_data['pesos_criterios']

# Criar DataFrame para visualização
df_scores = pd.DataFrame({
    'Cidade': list(scores.keys()),
    'Score TOPSIS': list(scores.values())
})

# Gráfico de barras dos scores
fig = px.bar(df_scores, x='Cidade', y='Score TOPSIS', 
             title='Scores MCDA-TOPSIS: Recife vs Salvador',
             color='Score TOPSIS',
             color_continuous_scale='viridis')

fig.update_layout(height=400, showlegend=False)
fig.show()

print(f"🏆 Vencedor MCDA: {mcda_data['melhor_alternativa']}")
print(f"📊 Confiança: {mcda_data['confianca_decisao']}")

## 2. Análise de Critérios e Pesos

In [ ]:
# Visualizar pesos dos critérios
df_pesos = pd.DataFrame({
    'Critério': list(pesos.keys()),
    'Peso': list(pesos.values())
})

# Mapeamento de nomes mais amigáveis
nome_criterios = {
    'custo_imobiliario': 'Custo Imobiliário',
    'tempo_entrega': 'Tempo de Entrega',
    'potencial_mercado': 'Potencial de Mercado',
    'infraestrutura': 'Infraestrutura',
    'tco_operacional': 'TCO Operacional'
}

df_pesos['Critério_Nome'] = df_pesos['Critério'].map(nome_criterios)

# Gráfico de pizza dos pesos
fig = px.pie(df_pesos, values='Peso', names='Critério_Nome',
             title='Distribuição de Pesos dos Critérios MCDA')
fig.show()

# Tabela resumo
print("\n📋 Resumo dos Critérios:")
display(df_pesos[['Critério_Nome', 'Peso']].round(3))

## 3. Simulação Monte Carlo - Análise de Riscos

In [ ]:
# Extrair dados Monte Carlo
mc_data = resultados['monte_carlo']
simulacoes = mc_data['resultados_simulacao']

# Criar DataFrames para análise
tco_recife = simulacoes['recife']['tco_scenarios']
tco_salvador = simulacoes['salvador']['tco_scenarios']
roi_recife = simulacoes['recife']['roi_scenarios']
roi_salvador = simulacoes['salvador']['roi_scenarios']

# Distribuições de TCO
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=['Distribuição TCO', 'Distribuição ROI'])

# TCO
fig.add_trace(go.Histogram(x=tco_recife, name='Recife TCO', opacity=0.7), row=1, col=1)
fig.add_trace(go.Histogram(x=tco_salvador, name='Salvador TCO', opacity=0.7), row=1, col=1)

# ROI
fig.add_trace(go.Histogram(x=roi_recife, name='Recife ROI', opacity=0.7), row=1, col=2)
fig.add_trace(go.Histogram(x=roi_salvador, name='Salvador ROI', opacity=0.7), row=1, col=2)

fig.update_layout(height=400, title_text="Análise Monte Carlo: TCO e ROI")
fig.show()

# Estatísticas resumo
print("\n📊 Estatísticas Monte Carlo:")
stats_df = pd.DataFrame({
    'Métrica': ['TCO Médio (R$ Mi)', 'TCO Desvio (R$ Mi)', 'ROI Médio (%)', 'Prob. ROI Positivo'],
    'Recife': [np.mean(tco_recife), np.std(tco_recife), np.mean(roi_recife), np.mean(np.array(roi_recife) > 0)],
    'Salvador': [np.mean(tco_salvador), np.std(tco_salvador), np.mean(roi_salvador), np.mean(np.array(roi_salvador) > 0)]
})
display(stats_df.round(3))

## 4. Mapa de Cobertura Logística

In [ ]:
# Criar mapa interativo
import config

# Centro do mapa (Nordeste)
centro_lat, centro_lon = -9.5, -37.0
m = folium.Map(location=[centro_lat, centro_lon], zoom_start=6)

# Adicionar candidatos
folium.Marker(
    location=config.RECIFE_COORDS,
    popup='Recife - Candidato CD',
    icon=folium.Icon(color='blue', icon='warehouse', prefix='fa')
).add_to(m)

folium.Marker(
    location=config.SALVADOR_COORDS,
    popup='Salvador - Candidato CD',
    icon=folium.Icon(color='red', icon='warehouse', prefix='fa')
).add_to(m)

# Adicionar capitais
for cidade, coords in config.CAPITAIS_NORDESTE.items():
    folium.Marker(
        location=coords,
        popup=f'{cidade} - Ponto de Demanda',
        icon=folium.Icon(color='green', icon='city', prefix='fa')
    ).add_to(m)

# Círculos de cobertura (raio aproximado de 500km)
folium.Circle(
    location=config.RECIFE_COORDS,
    radius=500000,  # 500km
    popup='Área de Cobertura Recife',
    color='blue',
    fill=True,
    opacity=0.3
).add_to(m)

folium.Circle(
    location=config.SALVADOR_COORDS,
    radius=500000,  # 500km
    popup='Área de Cobertura Salvador',
    color='red',
    fill=True,
    opacity=0.3
).add_to(m)

# Salvar mapa
m.save('../results/mapa_cobertura_cd.html')
print("🗺️ Mapa de cobertura salvo em: results/mapa_cobertura_cd.html")

# Exibir mapa no notebook
m

## 5. Análise de Tempos de Entrega

In [ ]:
# Extrair dados de distância e tempo
dados = resultados['dados_coletados']

# Preparar dados para visualização
cidades_destino = []
tempos_recife = []
tempos_salvador = []
distancias_recife = []
distancias_salvador = []

for cidade in config.CAPITAIS_NORDESTE.keys():
    cidades_destino.append(cidade)
    tempos_recife.append(dados['recife']['distancias'][cidade]['tempo_horas'])
    tempos_salvador.append(dados['salvador']['distancias'][cidade]['tempo_horas'])
    distancias_recife.append(dados['recife']['distancias'][cidade]['distancia_km'])
    distancias_salvador.append(dados['salvador']['distancias'][cidade]['distancia_km'])

# Criar DataFrame
df_tempos = pd.DataFrame({
    'Cidade': cidades_destino,
    'Tempo_Recife_h': tempos_recife,
    'Tempo_Salvador_h': tempos_salvador,
    'Distancia_Recife_km': distancias_recife,
    'Distancia_Salvador_km': distancias_salvador
})

# Gráfico comparativo de tempos
fig = go.Figure()
fig.add_trace(go.Bar(name='Recife', x=df_tempos['Cidade'], y=df_tempos['Tempo_Recife_h']))
fig.add_trace(go.Bar(name='Salvador', x=df_tempos['Cidade'], y=df_tempos['Tempo_Salvador_h']))

fig.update_layout(
    title='Comparação de Tempos de Entrega por Cidade',
    xaxis_title='Cidade de Destino',
    yaxis_title='Tempo de Entrega (horas)',
    barmode='group'
)
fig.show()

# Estatísticas resumo
print("\n⏰ Resumo de Tempos de Entrega:")
resumo_tempos = pd.DataFrame({
    'Métrica': ['Tempo Médio (h)', 'Tempo Máximo (h)', 'Tempo Mínimo (h)'],
    'Recife': [np.mean(tempos_recife), np.max(tempos_recife), np.min(tempos_recife)],
    'Salvador': [np.mean(tempos_salvador), np.max(tempos_salvador), np.min(tempos_salvador)]
})
display(resumo_tempos.round(2))

## 6. Consolidação Final e Recomendação

In [ ]:
# Extrair recomendação final
rec_final = resultados['recomendacao_final']

# Visualizar sistema de votação
votos = rec_final['votos_detalhados']
df_votos = pd.DataFrame({
    'Cidade': list(votos.keys()),
    'Score_Total': list(votos.values())
})

fig = px.bar(df_votos, x='Cidade', y='Score_Total',
             title='Sistema de Votação Ponderada - Score Final',
             color='Score_Total',
             color_continuous_scale='RdYlGn')
fig.show()

# Resumo executivo
print("\n" + "="*60)
print("🏆 RECOMENDAÇÃO FINAL")
print("="*60)
print(f"Cidade Recomendada: {rec_final['cidade_recomendada']}")
print(f"Score Final: {rec_final['score_final']}/100")
print(f"Confiança: {rec_final['confianca_consolidada']}")

print("\n📋 Justificativas:")
for i, justificativa in enumerate(rec_final['justificativas'], 1):
    print(f"{i}. {justificativa}")

print("\n🎯 Fatores Decisivos:")
for fator in rec_final['fatores_decisivos']:
    print(f"• {fator}")

## 7. Análise de Sensibilidade

In [ ]:
# Análise de sensibilidade dos pesos MCDA
sensibilidade = resultados['mcda']['analise_sensibilidade']

# Plotar análise de sensibilidade para cada critério
fig = make_subplots(rows=2, cols=3, 
                    subplot_titles=list(sensibilidade.keys()))

row_col_pairs = [(1,1), (1,2), (1,3), (2,1), (2,2)]

for i, (criterio, dados_sens) in enumerate(sensibilidade.items()):
    if i < len(row_col_pairs):
        row, col = row_col_pairs[i]
        
        variacoes = [d['variacao'] for d in dados_sens]
        scores_recife = [d['scores']['Recife'] for d in dados_sens]
        scores_salvador = [d['scores']['Salvador'] for d in dados_sens]
        
        fig.add_trace(go.Scatter(x=variacoes, y=scores_recife, name=f'Recife_{criterio}', 
                                line=dict(color='blue')), row=row, col=col)
        fig.add_trace(go.Scatter(x=variacoes, y=scores_salvador, name=f'Salvador_{criterio}', 
                                line=dict(color='red')), row=row, col=col)

fig.update_layout(height=600, title_text="Análise de Sensibilidade - Variação dos Pesos MCDA")
fig.show()

print("\n📊 A análise de sensibilidade mostra como mudanças nos pesos dos critérios")
print("afetam os scores finais de cada cidade.")

## 8. Exportar Resultados

In [ ]:
# Criar tabela resumo final
resumo_final = pd.DataFrame({
    'Metodologia': ['MCDA-TOPSIS', 'Monte Carlo', 'P-Mediana', 'Qualitativo'],
    'Recomendação': [
        resultados['mcda']['melhor_alternativa'],
        resultados['monte_carlo']['analise_comparativa']['recomendacao_risco'],
        resultados['preditiva']['recomendacao_otimizacao'],
        'Recife'  # Baseado na análise qualitativa
    ],
    'Peso_Decisão': ['30%', '25%', '35%', '10%'],
    'Confiança': [
        resultados['mcda']['confianca_decisao'],
        'Alta' if resultados['monte_carlo']['analise_comparativa']['prob_recife_menor_tco'] > 0.6 else 'Média',
        'Alta',
        'Média'
    ]
})

print("\n📊 Resumo por Metodologia:")
display(resumo_final)

# Salvar resumo
resumo_final.to_csv('../results/resumo_metodologias.csv', index=False)
df_tempos.to_csv('../results/analise_tempos_entrega.csv', index=False)

print("\n✅ Dados exportados para:")
print("• results/resumo_metodologias.csv")
print("• results/analise_tempos_entrega.csv")
print("• results/mapa_cobertura_cd.html")